In [ ]:
from import_phot import *

#Query Simbad: input=lista target, output= file con coordinate target (coo.txt)
first_query_gaia(r'C:\Users\Vito\Desktop\PhD\Progetti\BEAST\Stellar_ages\CMS\BT-Settl_stilism_cs0_GaiaDR3\target_list.txt')

#1st query VizieR: input=coo.txt, output= file con info target da Gaia DR3 (py_target_data.csv)
import_phot(r'C:\Users\Vito\Desktop\PhD\Progetti\BEAST\Stellar_ages\CMS\BT-Settl_stilism_cs0_GaiaDR3\coo.txt',r'C:\Users\Vito\Desktop\PhD\Progetti\BEAST\Stellar_ages\CMS\BT-Settl_stilism_cs0_GaiaDR3\py_target_data.csv',mode='Gaia_EDR3')

#Moduli da scrivere: input=py_target_data.csv, output=coordinate stelle comoving (CMS) (all_cms2.txt)
#------------- scrivere programma

#2nd query VizieR: input=all_cms2.txt, output=fotometria CMS Gaia e 2MASS
import_phot(r'C:\Users\Vito\Desktop\PhD\Progetti\BEAST\Stellar_ages\CMS\BT-Settl_stilism_cs0_GaiaDR3\all_cms2.txt',mode='all')